# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
from IPython.display import display

# Importação dos Dados

In [2]:
df = pd.read_csv('NPS_Complete_UFF_2022-07-18_2022-12-31.csv', sep=';', thousands = '.', decimal = ',')

# Pré-Tratamento

### Informações Irrelevantes


In [3]:
df = df.drop(['ref_month', 'CONTACT_DT'], axis=1) # Datas (irrelevante)
df = df.drop(['treatment_nm'], axis=1) # Muitas categorias possíveis que não sabemos o que significa ao certo
df = df.drop(['term_class'], axis=1) # Muitas categorias possíveis que estão mal divididas 

### Valores Nulos

In [4]:
MIN_NON_NULL = len(df) * 0.8 # Definindo um limite mínimo de valores não nulos para cada coluna como 80% do total (de linhas)
df = df.dropna(axis=1, thresh=MIN_NON_NULL) # Excluindo colunas que excedem o máximo de valores nulos

# ======== Tratando restantes =========

# FLOAT

# Nas colunas do tipo float, substituir valores nulos pela mediana (mais robustez contra outliers do que a média) 
for col in df.columns:
    if df[col].dtype == 'float64':
        df[col] = df[col].fillna(df[col].median())

# CATEGÓRICAS (casos especiais)

# USERVALUEQ2_VALUE
df['USERVALUEQ2_VALUE'] = df['USERVALUEQ2_VALUE'].str.replace(r'\d+\. ', '', regex=True) # Ajuste de ortografia
df['USERVALUEQ2_VALUE'] = df['USERVALUEQ2_VALUE'].str.replace(r'\d+\.', '', regex=True) # Ajuste de ortografia
df['USERVALUEQ2_VALUE'] = df['USERVALUEQ2_VALUE'].fillna('Não Informado') # Substitui nulos por 'Não Informado'

# term_fabr
df['term_fabr'] = df['term_fabr'].fillna('não informado') # Substitui nulos por 'não informado'
marcas = df['term_fabr'].value_counts()[df['term_fabr'].value_counts() < 200].index # Marcas com menos de 200 ocorrências
df['term_fabr'] = df['term_fabr'].replace(marcas, 'outros') # Substitui marcas com menos de 200 ocorrências por 'outros'

# device_tech
df = df.dropna(subset=['device_tech']) # Exclui linhas nulas

### Valores Únicos

In [5]:
# Existem colunas totalmente preenchidas pelo mesmo valor (independentemente de qual valor este seja)? Se sim, excluir
for col in df.columns:
    if len(df[col].unique()) == 1:
        df = df.drop(col, axis=1)

### Outliers [atenção aqui]

In [6]:
# Visualizando outliers para todas as colunas de tipo numérico usando o .describe()
'''for col in df.columns:
    if df[col].dtype == 'float64':
        display(df[col].describe())'''

# nomes das colunas que são float
float_cols = [col for col in df.columns if df[col].dtype == 'float64']
print(float_cols)

['vol_Rede_Social', 'vol_Video', 'vol_Comunicacao', 'vol_Loja_de_Apps', 'vol_Musica', 'vol_Google', 'vol_Navegacao', 'vol_InternetBank', 'vol_Transporte', 'vol_Jogos', 'vol_MarketPlace', 'vol_TIM', 'vol_Alimentacao', 'vol_Ecommerce', 'vol_Viagem', 'tot_duracao_chamada', 'originada_perc', 'eficiencia_Voz', 'tot_volume', 'vol_gratuito_perc', 'vol_4g_perc', 'vol_ul_perc', 'eficiencia_pdp', 'qtde_municipio', 'VoLTE_perc', 'DISP_STATUS_p', 'ACD_STATUS_p', 'OCUP_STATUS_p', 'dias_franquia_zerada', 'dias_traf_red', 'dias_consumo_bonus_ctrl']


In [7]:
# não há outliers aparentes no limite inferior 
# há outliers aparentes no limite superior

'''# calculando o limite superior para cada coluna
for col in df.columns:
    if df[col].dtype == 'float64':
        q1 = df[col].quantile(0.25)
        q3 = df[col].quantile(0.75)
        iqr = q3 - q1
        upper_limit = q3 + 12 * iqr # limite já foi muito elevado
        # exibindo quantidade de outliers em cada coluna
        print(f'Coluna {col} possui {len(df[df[col] > upper_limit])} outliers.')'''

"# calculando o limite superior para cada coluna\nfor col in df.columns:\n    if df[col].dtype == 'float64':\n        q1 = df[col].quantile(0.25)\n        q3 = df[col].quantile(0.75)\n        iqr = q3 - q1\n        upper_limit = q3 + 12 * iqr # limite já foi muito elevado\n        # exibindo quantidade de outliers em cada coluna\n        print(f'Coluna {col} possui {len(df[df[col] > upper_limit])} outliers.')"

Muitos outliers apontados... o que vai contra a ideia de outlier

Além disso, não temos unidades de medida, portanto, não podemos afirmar que valores são erros de medida. 

Dessa forma, vamos aplicar uma transformação logarítmica como forma de reduzir a influência dos outliers - caso existam.

Entretanto, a transformação logarítmica mitigaria a análise exploratória desses dados. Sendo assim, não será feita aqui. Será abordada em um tratamento extra antes da etapa de predição.

### Valores infinitos

In [8]:
# Verifica se há colunas com valores infinitos na coluna vol_Rede_Social
isinf = df['vol_Rede_Social'].apply(np.isinf)
isinf.value_counts()

False    45206
Name: vol_Rede_Social, dtype: int64

In [9]:
# Eliminar linhas com valores infinitos
#df = df[~isinf]

# Exportar df Final

In [10]:
display(df)

,ANF,CUSTOMER_SEGMENT,SURVEY,USERVALUEQ1_LABEL,USERVALUEQ2_VALUE,NPS_Class,vol_Rede_Social,vol_Video,vol_Comunicacao,vol_Loja_de_Apps,...,VoLTE_perc,term_fabr,device_tech,DISP_STATUS_p,ACD_STATUS_p,OCUP_STATUS_p,Qtd_CRM,dias_franquia_zerada,dias_traf_red,dias_consumo_bonus_ctrl
0,51,Pós-pago,Uso Dados,8,Não Informado,Passivo,3331.0,535.0,363.0,210.0,...,86.0,lg,4G,0.000000,0.000000,0.000000,0,0.0,0.0,0.000000e+00
1,11,Pré-pago,Uso Dados,9,Plano/Bonus,Promotor,1314.0,173.0,197.0,7.0,...,91.0,samsung,4G,0.164670,0.000000,0.056800,0,0.0,0.0,0.000000e+00
2,37,Pré-pago,Uso Dados,10,Velocidade,Promotor,3276.0,346.0,798.0,18.0,...,63.0,motorola,4G,0.799740,0.000000,1.327936,0,0.0,0.0,0.000000e+00
3,81,Pré-pago,Uso Dados,10,Outros,Promotor,134.0,321.0,4.0,0.0,...,0.0,outros,3G,0.387239,0.040754,0.000000,0,0.0,0.0,0.000000e+00
4,85,Pré-pago,Uso Dados,10,Sinal/Cobertura,Promotor,44.0,31.0,372.0,6.0,...,0.0,não informado,4G,0.227265,0.000000,0.000000,0,0.0,0.0,0.000000e+00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46635,48,Controle,Uso Dados,0,Sinal/Cobertura,Detrator,193.0,221.0,46.0,160.0,...,0.0,xiaomi,4G,0.000000,0.000000,0.552183,0,0.0,0.0,0.000000e+00
46636,35,Controle,Uso Dados,7,Sinal/Cobertura,Passivo,2918.0,426.0,2336.0,50.0,...,0.0,samsung,4G,0.000000,0.000000,0.000000,0,0.0,0.0,2.258065e+16
46637,43,Controle,Uso Dados,10,Duracao pacote dados,Promotor,31.0,207.0,70.0,4.0,...,100.0,motorola,5G,0.226444,0.000000,0.000000,0,0.0,0.0,0.000000e+00
46638,42,Controle,Uso Dados,10,Sinal/Cobertura,Promotor,0.0,174.0,4.0,0.0,...,0.0,samsung,3G,0.000000,0.000000,0.000000,0,0.0,0.0,0.000000e+00


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45206 entries, 0 to 46639
Data columns (total 47 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   ANF                      45206 non-null  int64  
 1   CUSTOMER_SEGMENT         45206 non-null  object 
 2   SURVEY                   45206 non-null  object 
 3   USERVALUEQ1_LABEL        45206 non-null  int64  
 4   USERVALUEQ2_VALUE        45206 non-null  object 
 5   NPS_Class                45206 non-null  object 
 6   vol_Rede_Social          45206 non-null  float64
 7   vol_Video                45206 non-null  float64
 8   vol_Comunicacao          45206 non-null  float64
 9   vol_Loja_de_Apps         45206 non-null  float64
 10  vol_Musica               45206 non-null  float64
 11  vol_Google               45206 non-null  float64
 12  vol_Navegacao            45206 non-null  float64
 13  vol_InternetBank         45206 non-null  float64
 14  vol_Transporte        

In [12]:
# Salvar df em arquivo csv - Lembrando: sem transformação logarítmica - para a caracterização
df.to_csv('NPS_Complete_UFF_2022-07-18_2022-12-31_pre-tratado.csv', sep=';', index=False)